# A story of failures in the optimisation of Water Distribution Systems.

Aware of the computational needs of our idea (joint optimisation of WDS design and operation), I started the development of a C++ Library to work as an interface between an optimization library and a hydraulic simulation library.



In [ ]:
# Imports
import os
import sys
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as subp
import pandas as pd
import numpy as np

import pygmo as pg

sys.path.append('..')
import pybeme

In [ ]:
# Load the experiments, prepare the plots
experiments = pybeme.load_experiments(os.path.join('data', 'all'), verbose=False)
print(experiments.keys())

In [ ]:
# Plot the pareto front of the first round of experiments.
# There is something wrong, so plot the Head and pressure for the tanks, junctions
msize = 10
xlims = [-.5e6, 20.5e6]
ylims = [-0.05, 0.605]
xaxis_title_text='Cost [$]'
yaxis_title_text='OF Reliability Index f1 [-]'
title_font_size=20
legend_font_size=16
def_font_size=12
def fix_layout(a_fig: go.Figure, a_title: str) -> None:
    a_fig.update_layout(title=dict(
                            text=a_title,
                            xanchor='center',
                            x=0.5,
                            yanchor='top',
                            y=1,
                            font_size=title_font_size
                        ),
                        plot_bgcolor='white',
                        paper_bgcolor='white',
                        xaxis=dict(
                            title=xaxis_title_text,
                            range=xlims,
                            automargin=True,
                            showline=True,
                            showgrid=True,
                            linewidth=1,
                            linecolor='grey',
                            zerolinecolor='black',
                            gridcolor='lightgrey'
                        ),
                        yaxis=dict(
                            title=yaxis_title_text,
                            range=ylims,
                            automargin=True,
                            showline=True,
                            showgrid=True,
                            linewidth=1,
                            linecolor='grey',
                            zerolinecolor='black',
                            gridcolor='lightgrey'
                        ),
                        width=1200,
                        height=800,
                        font=dict(
                            family="Lato",
                            color="black",
                            size=def_font_size
                        ),
                        margin=dict(
                            l=10,
                            r=10,
                            b=10,
                            t=50,
                            pad=0
                        ),
                        showlegend=True,
                        legend=dict(
                            orientation="v",
                            xanchor="left",
                            x=0.03,  
                            yanchor="top",
                            y=0.9,  
                            itemsizing='trace',  # To ensure items in legend keep the same size
                            traceorder="normal",
                            bgcolor="White",  # Background color
                            bordercolor="Black",  # Border color
                            borderwidth=1,  # Border width
                            groupclick="toggleitem",
                            itemclick="toggleothers",
                            itemdoubleclick="toggle",
                            tracegroupgap=100,
                            font_size=legend_font_size
                        )
)

In [ ]:
exps2plot = ['nsga2__anytown_mixed_f1__exp02', 'nsga2__anytown_rehab_f1__exp04', 'nsga2__anytown_rehab_f1__fullpower', 'nsga2__anytown_rehab_f1__median']
colors = ['#29378A', '#808080', '#74BDA7','#02B9EA']
markers = ['circle', 'circle', 'square', 'diamond']
names = ['Integrated', 'Pure design - Siew et al.', 'Pure design - Full power', 'Pure design - Median pattern']

def plot_pareto_fronts(experiments: dict, exps2plot: list, colors: list, markers: list, names: list):
    fig = go.Figure()

    for e, expname, in enumerate(exps2plot):
        # Extract the fitness vector for all final individuals across all islands
        final_fvs = experiments[expname].final_fitness_vectors.to_numpy()

        # I want full color the best pareto front of each solution and a lighter color for the rest, which are still pareto fronts but for the individual islands
        # Also, I need to make transparent solutions in the best pareto front but that are not feasible (i.e. reliability index < 0)
        pf = pg.non_dominated_front_2d(final_fvs)
        pf = pf[final_fvs[pf,1] <= -0.1] # only feasible solutions (I should do a simulation but this will do)
        npf = np.setdiff1d(np.arange(final_fvs.shape[0]), pf)
        
        fig.add_trace(go.Scatter(x=final_fvs[pf,0], y=-final_fvs[pf,1], mode='markers',
                                marker=dict(size=msize, symbol=markers[e], color=colors[e]),  
                                showlegend=True, name=names[e],
                                legendgroup=expname,
                                customdata=np.array(pf, dtype=str),
                                hovertemplate='Cost: %{x:2.2f} <br> Reliability Index: %{y:.2f} <extra>%{customdata}</extra>'
                                ) )
        fig.add_trace(go.Scatter(x=final_fvs[npf,0], y=-final_fvs[npf,1], mode='markers',
                                marker=dict(size=msize, symbol=markers[e]+'-open', color=colors[e]),  
                                showlegend=True, name=names[e],
                                legendgroup=expname,
                                customdata=np.array(npf, dtype=str),
                                hovertemplate='Cost: %{x:2.2f} <br> Reliability Index: %{y:.2f} <extra>%{customdata}</extra>'
                                ) )

    fix_layout(fig, 'Pareto fronts of the different solutions')

    fig.show()

plot_pareto_fronts(experiments, exps2plot, colors, markers, names)

There is something wrong, we checked and I forgot to track the missing steps. This is not ok but it is the default behaviour with wntr...
with EPYT this is not hte the behaviour if u use getComputedHydraulicTimeSeries.

## v 24.06.0
We fixed the use of all the instances. now we have the correct reliability

In [ ]:
exps2plot = ['nsga2__anytown_mixed_f1__exp02', 'nsga2__anytown_mixed_f1__exp03', 'nsga2__anytown_rehab_f1__exp04', 'nsga2__anytown_rehab_f1__exp05']
colors = ['#29378A', '#29378A', '#808080', '#808080']
markers = ['circle', 'square', 'circle', 'square']
names = ['Integrated - v24.4.0', 'Integrated - v24.6.0', 'Pure design - v24.4.0', 'Pure design - v24.6.0']
plot_pareto_fronts(experiments, exps2plot, colors, markers, names)

fig = subp.make_subplots(rows=2, cols=2, subplot_titles=['v24.4.0 -> v24.6.0', 'v24.6.0 -> v24.6.0', 'v24.4.0 -> v24.4.0', 'v24.6.0 -> v24.4.0'],
                         shared_xaxes=True, shared_yaxes=True)

for e, expname in enumerate(exps2plot):
    final_fvs = experiments[expname].final_fitness_vectors
    fv_pre = final_fvs.to_numpy()
    pf = pg.non_dominated_front_2d(fv_pre)
    pf = pf[fv_pre[pf,1] <= -0.1]
    fv_pf = fv_pre[pf]
    
    final_indvs_coords = final_fvs.index[pf]
    fv_post_04 = np.zeros_like(fv_pf)
    fv_post_06 = np.zeros_like(fv_pf)

    for i, fic in enumerate(final_indvs_coords):
        final_indv_coord = (fic[0], # island name
                            experiments[expname].generations[fic[0]].to_numpy()[-1], # last generation of the island
                            fic[1]) # individual index

        sim = experiments[expname].simulator(final_indv_coord)
        
        for v, vers in enumerate(['v24.04.00', 'v24.06.00']):

            sim.data['bemelib_version'] = vers
            sim.run()

            if v == 0:
                fv_post_04[i] = sim.result
            else:
                fv_post_06[i] = sim.result
                
    # Now plot the differences
    if e == 0 or e == 2:
        col = 1
    else :
        col = 2

    delta_c_04 = fv_pf[:, 0] -fv_post_04[:, 0]
    delta_r_04 = fv_pf[:, 1] -fv_post_04[:, 1]
    delta_c_06 = fv_pf[:, 0] -fv_post_06[:, 0]
    delta_r_06 = fv_pf[:, 1] -fv_post_06[:, 1]

    # Percentage error, I NEED to keep the sign to check if we overestimate or underestimate
    pe_c_04= delta_c_04 / fv_pf[:, 0]
    pe_r_04 = delta_r_04 / -fv_pf[:, 1]
    pe_c_06 = delta_c_06 / fv_pf[:, 0]
    pe_r_06 = delta_r_06 / -fv_pf[:, 1]
    # Any reliability change greater in abs term than 1 is to consider a solution that was working that now is not working
    # We put them in -1.1 or percentage error 110%
    pe_r_04[np.abs(delta_r_04) > 1] = -1.1
    pe_r_06[np.abs(delta_r_06) > 1] = -1.1

    
    fig.add_trace(go.Scatter(x= pe_c_04, y= pe_r_04, mode='markers',
                            marker=dict(size=msize, symbol=markers[e], color=colors[e]),  
                            showlegend=False,
                            customdata=np.array(np.arange(fv_pre.shape[0]), dtype=str),
                            hovertemplate='Cost: %{x:2.2f} <br> Reliability Index: %{y:.2f} <extra>%{customdata}</extra>'
                            ), row=2, col=col)
    fig.add_trace(go.Scatter(x= pe_c_06, y= pe_r_06, mode='markers',
                            marker=dict(size=msize, symbol=markers[e], color=colors[e]),  
                            showlegend=False,
                            customdata=np.array(np.arange(fv_pre.shape[0]), dtype=str),
                            hovertemplate='Cost: %{x:2.2f} <br> Reliability Index: %{y:.2f} <extra>%{customdata}</extra>'
                            ), row=1, col=col)
    

    
# For all 4 subplots, fix the xrange and yrange between the calculated max and min
for i in range(1, 3):
    for j in range(1, 3):
        fig.update_xaxes(range=[-0.2, 0.2], row=i, col=j)
        fig.update_yaxes(range=[-1.2, 1.2], row=i, col=j)
fig.show()


Ok but still not happy so we decided to do shorter hydraulics because we were experimenting the same behavoiur 

In [ ]:
exps2plot = ['nsga2__anytown_mixed_f1__exp03', 'nsga2__anytown_mixed_f1__exp04', 'nsga2__anytown_mixed_f1__exp05',
             'nsga2__anytown_rehab_f1__exp05', 'nsga2__anytown_rehab_f1__exp06', 'nsga2__anytown_rehab_f1__exp07']
colors = ['#29378A', '#29378A', '#29378A',
          '#808080', '#808080', '#808080']
markers = ['circle', 'square', 'diamond',
           'circle', 'square', 'diamond']
names = ['Integrated - 60min', 'Integrated - 30min', 'Integrated - 15min',
        'Pure design - 60min', 'Pure design - 30min', 'Pure design - 15min']
plot_pareto_fronts(experiments, exps2plot, colors, markers, names)

fig = subp.make_subplots(rows=3, cols=3, subplot_titles=['15 min -> 60 min', '30 min -> 60 min', '60 min -> 60 min',
                                                         '15 min -> 30 min', '30 min -> 30 min', '60 min -> 30 min',
                                                        '15 min -> 15 min', '30 min -> 15 min', '60 min -> 15 min'],
                            shared_xaxes=True, shared_yaxes=True)

for e, expname in enumerate(exps2plot):
    final_fvs = experiments[expname].final_fitness_vectors
    fv_pre = final_fvs.to_numpy()
    pf = pg.non_dominated_front_2d(fv_pre)
    pf = pf[fv_pre[pf,1] <= -0.1]
    fv_pf = fv_pre[pf]
    
    final_indvs_coords = final_fvs.index[pf]
    fv_post15 = np.zeros_like(fv_pf)
    fv_post30 = np.zeros_like(fv_pf)
    fv_post60 = np.zeros_like(fv_pf)

    for i, fic in enumerate(final_indvs_coords):
        final_indv_coord = (fic[0], # island name
                            experiments[expname].generations[fic[0]].to_numpy()[-1], # last generation of the island
                            fic[1]) # individual index

        sim = experiments[expname].simulator(final_indv_coord)
        
        for m, min in enumerate([15, 30, 60]):
            
            sim.run()

            if v == 0:
                fv_post_04[i] = sim.result
            else:
                fv_post_06[i] = sim.result
                
    # Now plot the differences
    if e == 0 or e == 2:
        col = 1
    else :
        col = 2

    delta_c_04 = fv_pf[:, 0] -fv_post_04[:, 0]
    delta_r_04 = fv_pf[:, 1] -fv_post_04[:, 1]
    delta_c_06 = fv_pf[:, 0] -fv_post_06[:, 0]
    delta_r_06 = fv_pf[:, 1] -fv_post_06[:, 1]

    # Percentage error, I NEED to keep the sign to check if we overestimate or underestimate
    pe_c_04= delta_c_04 / fv_pf[:, 0]
    pe_r_04 = delta_r_04 / -fv_pf[:, 1]
    pe_c_06 = delta_c_06 / fv_pf[:, 0]
    pe_r_06 = delta_r_06 / -fv_pf[:, 1]
    # Any reliability change greater in abs term than 1 is to consider a solution that was working that now is not working
    # We put them in -1.1 or percentage error 110%
    pe_r_04[np.abs(delta_r_04) > 1] = -1.1
    pe_r_06[np.abs(delta_r_06) > 1] = -1.1

    
    fig.add_trace(go.Scatter(x= pe_c_04, y= pe_r_04, mode='markers',
                            marker=dict(size=msize, symbol=markers[e], color=colors[e]),  
                            showlegend=False,
                            customdata=np.array(np.arange(fv_pre.shape[0]), dtype=str),
                            hovertemplate='Cost: %{x:2.2f} <br> Reliability Index: %{y:.2f} <extra>%{customdata}</extra>'
                            ), row=2, col=col)
    fig.add_trace(go.Scatter(x= pe_c_06, y= pe_r_06, mode='markers',
                            marker=dict(size=msize, symbol=markers[e], color=colors[e]),  
                            showlegend=False,
                            customdata=np.array(np.arange(fv_pre.shape[0]), dtype=str),
                            hovertemplate='Cost: %{x:2.2f} <br> Reliability Index: %{y:.2f} <extra>%{customdata}</extra>'
                            ), row=1, col=col)
    

    
# For all 4 subplots, fix the xrange and yrange between the calculated max and min
for i in range(1, 3):
    for j in range(1, 3):
        fig.update_xaxes(range=[-0.2, 0.2], row=i, col=j)
        fig.update_yaxes(range=[-1.2, 1.2], row=i, col=j)
fig.show()